In [1]:
#Librerias usadas
import numpy as np #Manejo de arreglos
import matplotlib.pyplot as plt #Dibujar las matrices
import tensorflow as tf #Redes neuronales
import cv2 #Manejo de imagenes
import keras #Estructura de redes neuronales
import math #Funciones matematicas

import os#Creacion de directorios(operating system)
import pickle #manejar archivos y listas

In [2]:
import tkinter as tk
from tkinter import * #Libreria para hacer interfaces graficas en python
from tkinter import messagebox
from tkinter import ttk #Libreria para hacer botones
from wand.image import Image   #Libreria para convertir un formato postscript a bmp
import math

In [3]:
#Creacion de directorios
folder='Interfaz grafica Deltas'
user='Monroy969'
    
#Cargamos diccionario
diccionario = [] 
a=open('C:/Users/'+user+'/Desktop/'+folder+'/Diccionario/ListaPalabrasEsp_Sorted-2.txt', 'r', encoding='utf-8')

for linea in a:
    diccionario.append(linea)#Se leen las palabras
a.close

#Quitamos los escapes de linea
for x in range(len(diccionario)):
    diccionario[x]=diccionario[x].replace("\n","")

#Determinamos el tamaño del diccionario 
lendcn=len(diccionario) #Determinamos el tamaño del diccionario
exp=0

#Se calcula la diferencia para qe el tamaño del diccionaro sea en potencias de 2(busqueda log)
while(lendcn): 
    lendcn>>=1
    exp+=1
diferencia=2**exp-len(diccionario)

#Preprocesado del diccionario
for x in range(diferencia): #Rellenamos el diccionario con la palabra zzz para completar las potencias
    diccionario.append('zzz')#las zzz ayudan a que no haya ruido
    
#Funcion de busqueda logaritmica
def logs(palabra):
    resultados='' #Listas que gurda si la palabra se encuentra o no
    find=True
    for p in range(1): #Este for evalua cada palabra en la lista
        palabra=palabra #En cada ciclo se evalua una palabra
        listsize = len(diccionario)/2 #Se parte a la mitad el tamaño del diccionario
        pos = listsize #Se guarda la "posicion" de la palabra
        iteraciones=0
        
        while(int(listsize)): #El while se ejecuta mienrtas queden palabras por buscar en el diccionario
            if palabra > diccionario[int(pos)]: #Determina si se buscara de la mitad para arriba
                pos+=listsize/2
            if palabra<diccionario[int(pos)]:#Determina si se buscara de la mitad para abajo
                pos-=listsize/2
            listsize/=2  #Se reduce el diccionario a la mitad
            iteraciones+=1

        if palabra == diccionario[int(pos)]: #Si la palabra es igual a una del diccionario segun la busqueda 
            resultados=palabra#Se guarda el resultado y se presenta
            find=True
    #Si no se encuentra la palabra se manda un texto.
    if resultados=='':
        resultados='no se encontro la palabra'
        find=False
    return resultados,find

#Definimos el tamaño de las imagenes
Alto=28
Ancho=28

#Definimos las clases
clases = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']

#Activar 1 vez
#Cargamos el modelo preentrenado en googlecolab
model=keras.models.load_model('C:/Users/'+user+'/Desktop/'+folder+'/Modelos/emnist_cnn_model9638.h5')

In [4]:
numero_letras=0 #Contador de letras a usar
#Funcion que sirve con el evento de clic e inicializa el punto inicial del dibujo
def lastxy(event):
    global lastx,lasty #Variables globales que alamcenan las posiciones anteriores de (x,y)
    lastx,lasty=event.x,event.y #Determina la posicion donde se dio clic
    root.bind('<B1-Motion>',write) #Se llama ala funcion write en caso de que el raton se mueva despues de hacer clic


#Funcion para dibujar en el canvas
def write(event):
    global lastx,lasty #Usa las variables globales obtenidas de la funcion lastxy
    global numero_letras #Contador de letras totales
    global Letras
    xr, yr = abs(root.winfo_x()-event.x_root),abs(root.winfo_y()-event.y_root) #Determina la posicion absoluta del rarton en la raiz
    xc, yc = event.x, event.y#Determina la posicion del raton en el canvas
    #Funcion para delimitar,contar y separar los canvas
    cota_i=9
    cota_s=200
    if yr<=233:
        for x in range(len(Letras)):
            if xr>=cota_i and xr<=cota_s:
                Letras[x].create_line(lastx,lasty,xc,yc,width=20,fill='white',capstyle=ROUND,joinstyle=ROUND,smooth=True)
                lastx,lasty=xc,yc
                Banderas[x]=True
                if Banderas[x]!=Banderasa[x]:
                    numero_letras+=1
                Banderasa[x]=Banderas[x]
            cota_i+=195+17
            cota_s+=195+17
    else:
        None
        
def delete(event):
    global numero_letras
    xr, yr = abs(root.winfo_x()-event.x_root),abs(root.winfo_y()-event.y_root)
    xc, yc = event.x, event.y
    
    cota_i=5
    cota_s=190
    if yr<=233:
        for x in range(len(Letras)):
            if xr>=cota_i and xr<=cota_s:
                Letras[x].create_rectangle(0,0,width_c,height_c,fill = 'black')
                Banderas[x]=False
                if Banderas[x]!=Banderasa[x]:
                    numero_letras-=1
                Banderasa[x]=Banderas[x]
            cota_i+=210
            cota_s+=210

    else:
        None
        
def save():
    global numero_letras
    global Letras
    for x in range(numero_letras):
        ps = Letras[x].postscript(file = 'C:/Users/'+user+'/Desktop/'+folder+'/Tkinterps/'+ str(x) +'.ps',colormode = 'color')
        #Lee la imagen y la guarda en la variable img
        with Image(filename ='C:/Users/'+user+'/Desktop/'+folder+'/Tkinterps/'+ str(x) +'.ps') as img: 

            # Cambia el formato
            img.format = 'bmp'

            # Guarda la imagen
            img.save(filename ='C:/Users/'+user+'/Desktop/'+folder+'/Tkinterps/'+ str(x) +'.bmp')
            
def convertir():
    for x in range(numero_letras):
        letra = cv2.imread('C:/Users/'+user+'/Desktop/'+folder+'/Tkinterps/'+ str(x) +'.bmp', cv2.IMREAD_GRAYSCALE)
        letra =  cv2.GaussianBlur(letra,(3,3),0)
        resize=cv2.resize(letra, (28,28))
        resize =  cv2.GaussianBlur(resize,(3,3),0)
        cv2.imwrite('C:/Users/'+user+'/Desktop/'+folder+'/Tkinter/'+ str(x) +'.jpg',resize)
        
def predicciones():
    global numero_letras
    for numero_letra in range(numero_letras):
        #print(numero_letra)
        letra = cv2.imread('C:/Users/'+user+'/Desktop/'+folder+'/Tkinter/'+ str(numero_letra) +'.jpg', cv2.IMREAD_GRAYSCALE)

        #Definimos las dimensiones para la CNN (numero de imagenes,Alto,Ancho,Numero de canales de color)
        Datos_Predict = np.asarray(letra).reshape(-1,Alto,Ancho,1)
        Datos_Predict = Datos_Predict.astype('float32')

        #Evaluamos la prediccion
        y_pred = model.predict(Datos_Predict/255)#Evaluamos los datos

        #Se asigna la clase a los valores predichos
        letras=[]
        for n in range(len(y_pred)):
            aux=[]
            for j in range(len(y_pred[n])):
                #print(y_pred[n][j],clases[j])
                letras.append([y_pred[n][j],clases[j]]) #Asignacion de letra a cada exactitud

        #Ordenacion mayor a menor
        letras.sort(reverse=True)
        #Guardado de informacion
        with open('C:/Users/'+user+'/Desktop/'+folder+'/Tkinter/Prediccion'+str(numero_letra)+'.p', "wb") as f:
             pickle.dump(letras, f)

        with open('C:/Users/'+user+'/Desktop/'+folder+'/Tkinter/Prediccion'+str(numero_letra)+'.txt', "w") as f:
             f.write(str(letras)+'\n')
                
def construccion():
    global numero_letras
    deep=0
    palabra=""
    for numero_letra in range(numero_letras):
        with open('C:/Users/'+user+'/Desktop/'+folder+'/Tkinter/Prediccion'+str(numero_letra)+'.p', "rb") as f:
                obj=pickle.load(f)
                palabra=palabra+obj[deep][1]
    return palabra

def evaluacion(palabra):
    #Saber si la palabra tenia un error
    errores=0
    Errores=[]
    p,aciertos=logs(palabra)
    if aciertos==True:
        return palabra,aciertos
    else:
        return p,aciertos
    
def calculo_de_deltas(deep):
    deltas=[]
    #Extraccion de predicciones
    datos=[]
    for numero_letra in range(numero_letras):
        o=[]
        with open('C:/Users/'+user+'/Desktop/'+folder+'/Tkinter/Prediccion'+str(numero_letra)+'.p', "rb") as f: #Abrimos el archivo de predicciones
                obj=pickle.load(f) #obj es un arreglo con dos partes obj[orden de exactitud][letras o valor numérico]  

        #Paquete de predicciones
        for x in range(deep):
            o.append(obj[x])
        datos.append(o)

    #Ordenamiento de las deltas de menor a mayor
    deltas=[]
    salida=False
    x=deep-1
    while salida != True:
        for letras in range(len(datos)):
            #print(datos[letras][x-1],datos[letras][x])
            #print(abs(datos[letras][x-1][0]-datos[letras][x][0]),datos[letras][x][1])
            deltas.append(((abs(datos[letras][x-1][0]-datos[letras][x][0]),datos[letras][x][1]),letras))

        salida=True

    deltas.sort()
    return(deltas)

def construccion_palabras_delta(deep):
    cont=0
    deltas=calculo_de_deltas(deep)
    for posibilidad in range(len(deltas)):
        construccion=""
        i=0
        for numero_letra in range(numero_letras):
            #print(i,deltas[posibilidad][1])
            if i==deltas[posibilidad][1]:
                with open('C:/Users/'+user+'/Desktop/'+folder+'/Tkinter/Prediccion'+str(numero_letra)+'.p', "rb") as f:
                        obj=pickle.load(f) #obj es un arreglo con dos partes 
                                            #obj[orden de exactitud][letras o valor numérico]
                
                construccion= construccion+obj[deep-1][1]

            else:
                with open('C:/Users/'+user+'/Desktop/'+folder+'/Tkinter/Prediccion'+str(numero_letra)+'.p', "rb") as f:
                        obj=pickle.load(f) #obj es un arreglo con dos partes 
                                            #obj[orden de exactitud][letras o valor numérico]
                
                construccion= construccion+obj[0][1]
            i+=1

        l,estado=logs(construccion)
        cont+=1
        
        if estado==True:           
            p=l
            bandera=True
            break
    
    print(cont)
    return l,estado

def correccion_delta():
    deep=2
    iteraciones=0
    estado=False
    while estado==False:
        l,estado=construccion_palabras_delta(deep) #Errores,Deep,letra,palabras
        iteraciones+=1
        deep+=1

        if deep == 26:
            break
        
    if estado == False:
        return(l,estado)
    else:
        return l,estado
    return l,estado

def correccion2():
    #Correcion de 2 errores
    aciertos=False
    if aciertos==False:
        iteracion=0
        bandera=0
        deepc=1#Profundidad de exactitud
        for l in range(numero_letras):
            print(l)
            for deepc1 in range(1,26):
                for deepc2 in range(1,26):
                    for s in range(1,numero_letras):
                        palabra2=''
                        #print(l,s)
                        for numero_letra in range(numero_letras):
                            
                            with open('C:/Users/'+user+'/Desktop/'+folder+'/Tkinter/Prediccion'+str(numero_letra)+'.p', "rb") as f:
                                    obj=pickle.load(f)
                                    #print(numero_letra,il[Errores[numero_palabra]][0]+l ,il[Errores[numero_palabra]][0]+l+s )
                                    bandera=0
                                    if numero_letra==l:
                                        deep=deepc1
                                        palabra2=palabra2+obj[deep][1]
                                        #print('palabras cambiadas ',obj[deep][1])
                                        bandera=1

                                    if numero_letra==l+s:
                                        deep=deepc2
                                        palabra2=palabra2+obj[deep][1]
                                        #print('palabras cambiadas ',obj[deep][1])
                                        bandera=1

                                    if bandera!=1:
                                        deep=0
                                        palabra2=palabra2+obj[deep][1]
                                        #print('palabras no cambiadas ',obj[deep][1])
                        
                        resultado,aciertos=logs(palabra2)
                       
                        if aciertos==True: #Si resulta que se encontro la palabra se escribe la palabra y termina el ciclo
                            return resultado,aciertos
                            bandera=True
                            break
                    if bandera==True: #Salir del ciclo
                        break    
        return resultado,aciertos

def terminar():
    global numero_letras
    save()
    convertir()
    predicciones()
    palabra=construccion()
    print(palabra)
    resultado,aciertos=evaluacion(palabra)

    if aciertos==False:
        resultado,aciertos=correccion_delta()

    if aciertos==False:
        resultado,aciertos=correccion2()
        
    if aciertos==False:
        resultado=palabra
        
    messagebox.showinfo(message=resultado, title='Resultados')
    

#Crea la raiz
root = Tk()#nombre de la raiz
root.title("Reconocimiento de caracteres")#Titulo de la aplicacion

width_r = 1470 #Ancho de la ventana
height_r = 240 #Largo de la ventana

root.geometry(str(width_r)+'x'+str(height_r)+'+0+0') #Tamaño de la ventana
root.resizable(height = 0, width = 0) #Hacemos que no se pueda cambiar el tamaño de la ventana


#Tamaño del canvas
width_c = 200 #Ancho del canvas
height_c = 200 #Largo del canvas

#Lista de canvas
Letras=[None,None,None,None,None,None,None]
#Lista de banderas(para contar el numero de letras)
Banderas=[False,False,False,False,False,False,False]
Banderasa=[False,False,False,False,False,False,False]

for c in range(len(Letras)):
    xn=c*210 #Distancia entre cada canvas
    Letras[c]=Canvas(width=width_c, height=height_c, bg='blue',highlightthickness=0) #Creamos el canvas
    Letras[c].create_rectangle(0,0,width_c,height_c,fill = 'black') #Lo pintamos de negro
    Letras[c].place(x=xn, y=0)#Lo colocamos en las posiciones


boton1 = tk.Button(root, text="Analizar",width=100,height=2,command=terminar)
boton1.place(x=10, y=200)

boton2 = tk.Button(root, text="Salida",width=105,height=2,command=root.destroy)
boton2.place(x=715, y=200)
               
root.bind('<Button-1>',lastxy)
root.bind('<Button-3>',delete)

root.mainloop()

sol
